In [ ]:
!pip install opencv-python
!pip install gym
!pip install numpy
!pip install opencv-python-headless
!pip install tensorflow
!pip install matplotlib


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install gym[box2d]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for box2d-py
  Running setup.py clean for box2d-py
Failed to build box2d-py
  Running setup.py install for box2d-py ... done
  DEPRECATION: box2d-py was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. pip 23.1 will enforce this behaviour change. A possible replacement is to fix the wheel build issue repo

In [ ]:
import cv2
from gym import Env
import numpy as np
from gym.spaces import Discrete, Box
import random
import time
import matplotlib.pyplot as plt


class tbot_2d_env(Env):
	def get_obs_tuple(x, y, r):
		return []
	def __init__(self):
		self.action_space = Box(low=np.array([-1, -1]), high=np.array([1, 1])) # Actions v, w ranging from -1 to 1 (scaled later) 
		# Observation space of env, 12 lidar rays, angle and distance of agent to traget, linear and angular velocities of prev step 
		# self.observation_space = Box(low=np.array([0,0,0,0,0,0,0,0,0,0,0,0, -np.pi, 0, -1, -1]), high=np.array([1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5, np.pi, 5.7, 1, 1]))
		# Observation space of env, 12 lidar rays, x, y, theta pos of agent, x, y, theta pos of goal linear and angular velocities of prev step 
		self.observation_space = Box(low=np.array([0,0,0,0,0,0,0,0,0,0,0,0, 0.0, 0.0, -np.pi, 0.0, 0.0, -np.pi, -1, -1]), 
									 high=np.array([1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5, 4.0, 4.0, np.pi, 4.0, 4.0, np.pi,  1, 1]))
		# Initializing the robot's starting state, random lidar data, 12 lidar rays, x, y, theta pos of agent, x, y, theta pos of goal, linear and angular velocities of initial step
		self.state = np.array([1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5, 0.3, 0.3, 0, 3.3, 3.3, 0, 0, 0])
		self.ep_duration = 200
		self.obstacles = [np.array([1.5, 1.5, 0.2]), np.array([2.5, 1.5, 0.2]), np.array([2.5, 2.5, 0.2]), np.array([1.5, 2.5, 0.2])]
		self.img = np.ones((400,400,3), dtype=np.uint8)*255

	def random_start(self):
		self.state[12] = np.random.random()*0.3 + 0.3
		self.state[13] = np.random.random()*0.3 + 0.3
		self.state[14] = 0

	def random_goal(self):
		self.state[15] = np.random.random()*0.3 + 2.8
		self.state[16] = np.random.random()*0.3 + 2.8
		self.state[17] = 0

	def fixed_start(self):
		self.state[12] = 0.6
		self.state[13] = 0.6
		self.state[14] = 0

	def fixed_goal(self):
		self.state[15] = 3.3
		self.state[16] = 3.3
		self.state[17] = 0

	def reset(self):
		self.state = np.array([1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5, 0.3, 0.3, 0, 3.3, 3.3, 0, 0, 0])
		self.random_start()
		self.random_goal()
		# self.fixed_start()
		# self.fixed_goal()
		# self.state[15] = 0.8
		# self.state[16] = 1.1
		# self.state[17] = 0
		self.ep_duration = 200
		return self.state

	def step(self, action):
		r_i = 0.1 	# Radius of robot
		r_s = 0.4 	# Safety radius from obstacles
		reward = 0.0
		alpha = 0.5
		d_offset = 5
		done = False
		# blank info required by openAI gym
		info = {}

		d_to_goal = np.linalg.norm(self.state[12:14] - self.state[15:17])
		self.ep_duration -= 1
		# update state by performing action
		v = action[0]
		w = action[1]
		dt = 0.05

		# Updating the angle and distance of agent to target
		self.state[18] = v  	# Velocity in state
		self.state[19] = w  	# Angular velocity in state

		th = self.state[14]
		v = v*0.4

		for i in range(10):	
			self.state[12] += v*dt*np.cos(th)/10
			self.state[13] += v*dt*np.sin(th)/10
			th += w*dt/10

		self.state[14] = th
		# Read lidar data from new position after perfroming action
		# self.state [0:12] = get_lidar_data(self.state[12], self.state[13])
		self.get_lidar_data()

		# calculate reward
		# Checking distance from the goal position
		if d_to_goal < 0.25:
			reward += 200
			done = True
			return self.state, reward, done, info

		else:
			reward += alpha*(d_offset - d_to_goal)

		# Checking if robot moves out of the arena (4mx4m) area, giving a large negative reward and marking done if robot goes out of arena
		if self.state[12] < r_i or self.state[13] < r_i or self.state[12] > 4 - r_i or self.state[13] > 4 - r_i:
			reward -= 150
			done = True
			return self.state, reward, done, info

		
		for ray_scan in self.state[0:12]:
			if ray_scan < 0.4:
				reward -= 150
				done = True
				break

			elif ray_scan < (r_i + r_s) :
				reward += (ray_scan - 1.5)/(r_i + r_s)


		# if ep_duration <= 0, mark done
		if self.ep_duration <= 0 or done:
			done = True

		else:
			done = False

		# return updated state, reward, done flag, info
		return self.state, reward, done, info


	def get_lidar_data(self):
		self.state[0:12] = 1.5
		for obs in self.obstacles:
			d = np.linalg.norm(self.state[12:14] - obs[0:2]) 
			theta_rbt = self.state[14]
			rbt_x = self.state[12]
			rbt_y = self.state[13]
			if d > 1.5:
				continue

			theta = (np.arctan2(obs[1]-self.state[13], obs[0]-self.state[12]) - self.state[14])*180/np.pi
			if theta < 0:
				theta = 360 + theta

			theta = theta%360
			ray_id = int(theta//30)
			
			for i in range(3):
				b = 1
				m = (np.tan(theta_rbt + 30*(ray_id + i - 1)*np.pi/180))
				a = -m
				c = m*rbt_x - rbt_y
				p_dist = (np.abs(a*rbt_x + b*(rbt_y) + c))/np.sqrt(a**2 + b**2)
				if p_dist < obs[2]:
					# print(ray_id)
					self.state[(ray_id + i - 1)%12] = d - obs[2]


		# Use current robot position and orientation, obstacle position and shape to get the intersection point of ray with the obstacle

	def render(self):
		self.img = np.ones((400,400,3), dtype=np.uint8)*255
		cv2.circle(self.img, (int(self.state[12]*100), int(400-self.state[13]*100)), int(0.1*100), (255, 0, 0), thickness=-1)
		cv2.circle(self.img, (int(self.state[15]*100), int(400-self.state[16]*100)), int(0.1*100), (0, 255, 0), thickness=-1)
		for obs in self.obstacles:
			cv2.circle(self.img, (int(obs[0]*100), int(400-obs[1]*100)), int(obs[2]*100), (120, 120, 120), thickness=-1)

		return self.img
		# cv2.imshow("Enviroment window", self.img)



In [ ]:
# Importing all necessary libraries
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import os

# The replay buffer class
class replay_buffer:
	# Initialize the memory buffer. The size represents the number of individual experiences, 
	# mem_cntr keeps track of the replay index and prevents out-of-index errors
	# state mem stores the observations of current time step
	# new state mem stores the observations of next time step after action a has been performed (the transition)
	# The action and reward mem store the action and reward corresponding to that particular time step 
	# Terminal mem stores the if a particular time step was the terminal step (i.e. if the episode was marked done at that step)
	def __init__(self, max_size, input_shape, n_actions):
		self.mem_size = max_size
		self.mem_cntr = 0
		self.state_mem = np.zeros((self.mem_size, *input_shape))
		self.new_state_mem = np.zeros((self.mem_size, *input_shape))
		self.action_mem = np.zeros((self.mem_size, n_actions))
		self.reward_mem = np.zeros(self.mem_size)
		self.terminal_mem = np.zeros(self.mem_size, dtype=np.bool_)


	def store_in_buffer(self, state, action, reward, state_new, done):
		idx = self.mem_cntr % self.mem_size

		self.state_mem[idx] = state
		self.new_state_mem[idx] = state_new
		self.action_mem[idx] = action
		self.reward_mem[idx] = reward
		self.terminal_mem[idx] = done

		self.mem_cntr += 1

	def sample_buffer(self, batch_size):
		max_mem = min(self.mem_cntr, self.mem_size)

		batch = np.random.choice(max_mem, batch_size)

		states = self.state_mem[batch]
		new_states = self.new_state_mem[batch]
		actions = self.action_mem[batch]
		rewards = self.reward_mem[batch]
		dones = self.terminal_mem[batch]

		return states, actions, rewards, new_states, dones


class actor_nw(keras.Model):
	def __init__(self, fc1_dims, fc2_dims, fc3_dims, n_actions, name, chkpt_dir='tmp/td3'):
		super(actor_nw, self).__init__()
		self.layer1_dims = fc1_dims
		self.layer2_dims = fc2_dims
		self.layer3_dims = fc3_dims
		self.n_actions = n_actions
		self.model_name = name
		self.check_pt_dir = chkpt_dir
		self.check_pt_file = os.path.join(self.check_pt_dir, name+'_td3')

		self.layer1 = Dense(self.layer1_dims, activation='relu')
		self.layer2 = Dense(self.layer2_dims, activation='relu')
		self.layer3 = Dense(self.layer3_dims, activation='relu')
		self.mu = Dense(self.n_actions, activation='tanh')
	
	def call(self, state):
		actor_op = self.layer1(state)
		actor_op = self.layer2(actor_op)
		actor_op = self.layer3(actor_op)

		mu = self.mu(actor_op)

		return mu

class critic_nw(keras.Model):
	def __init__(self, fc1_dims, fc2_dims, fc3_dims, name, chkpt_dir='tmp/td3'):
		super(critic_nw, self).__init__()
		self.layer1_dims = fc1_dims
		self.layer2_dims = fc2_dims
		self.layer3_dims = fc3_dims
		self.model_name = name
		self.check_pt_dir = chkpt_dir
		self.check_pt_file = os.path.join(self.check_pt_dir, name+'_td3')

		self.layer1 = Dense(self.layer1_dims, activation='relu')
		self.layer2 = Dense(self.layer2_dims, activation='relu')
		self.layer3 = Dense(self.layer3_dims, activation='relu')
		self.q_op = Dense(1, activation=None)

	def call(self, state, action):
		q_value = self.layer1(tf.concat([state, action], axis=1))
		q_value = self.layer2(q_value)
		q_value = self.layer3(q_value)

		q = self.q_op(q_value)

		return q

# The agent class
class Agent:
	def __init__(self, agent_lr, critic_lr, input_dims, tau, env,
				 gamma=0.99, update_actor_interval=2, warmup=100,
				 n_actions=2, max_size=1000000, layer1_size=512,
				 layer2_size=256, layer3_size=128, batch_size=100, noise=0.1):
		self.gamma = gamma
		self.tau = tau
		self.max_action = env.action_space.high[0]
		self.min_action = env.action_space.low[0]
		self.mem = replay_buffer(max_size, input_dims, n_actions)
		self.batch_size = batch_size
		self.learn_step_cntr = 0
		self.time_step = 0
		self.warmup = warmup
		self.n_actions = n_actions
		self.update_actor_iter = update_actor_interval
		self.ep_greedy = 0.0

		self.actor = actor_nw(layer1_size, layer2_size, layer3_size, n_actions=n_actions, name='actor')
		self.critic_1 = critic_nw(layer1_size, layer2_size, layer3_size, name='critic_1')
		self.critic_2 = critic_nw(layer1_size, layer2_size, layer3_size, name='critic_2')

		self.target_actor = actor_nw(layer1_size, layer2_size, layer3_size, n_actions=n_actions, name='target_actor')
		self.target_critic_1 = critic_nw(layer1_size, layer2_size, layer3_size, name='target_critic_1')
		self.target_critic_2 = critic_nw(layer1_size, layer2_size, layer3_size, name='target_critic_2')

		self.actor.compile(optimizer=Adam(learning_rate=agent_lr), loss='mean')
		self.critic_1.compile(optimizer=Adam(learning_rate=critic_lr), loss='mean_squared_error')
		self.critic_2.compile(optimizer=Adam(learning_rate=critic_lr), loss='mean_squared_error')

		self.target_actor.compile(optimizer=Adam(learning_rate=agent_lr), loss='mean')
		self.target_critic_1.compile(optimizer=Adam(learning_rate=critic_lr), loss='mean_squared_error')
		self.target_critic_2.compile(optimizer=Adam(learning_rate=critic_lr), loss='mean_squared_error')

		self.noise = noise
		self.update_network_parameters(tau=1)

	def nw_action(self, observation):
		if self.time_step < self.warmup:
			mu = np.random.normal(scale=self.noise, size=(self.n_actions,))
		else:
			state = tf.convert_to_tensor([observation], dtype=tf.float32)
			# returns a batch size of 1, want a scalar array
			mu = self.actor(state)[0]

# 		if np.random.rand() > self.ep_greedy:
# 			# Pick random action
# 			mu_ = (tf.random.uniform(shape=[2]) - tf.constant([0.5, 0.5]))*2
# 			# tf.convert_to_tensor(np.array([(np.random.rand()-0.5)*2, (np.random.rand()-0.5)*2]))

# 		else:
		mu_ = mu + np.random.normal(scale=self.noise)
		mu_ = tf.clip_by_value(mu_, self.min_action, self.max_action)
		

		self.time_step += 1


		return mu_

	def remember(self, state, action, reward, new_state, done):
		self.mem.store_in_buffer(state, action, reward, new_state, done)

	def learn(self):
		if self.mem.mem_cntr < self.batch_size:
			return

		states, actions, rewards, new_states, dones = self.mem.sample_buffer(self.batch_size)

		states = tf.convert_to_tensor(states, dtype=tf.float32)
		actions = tf.convert_to_tensor(actions, dtype=tf.float32)
		rewards = tf.convert_to_tensor(rewards, dtype=tf.float32)
		states_ = tf.convert_to_tensor(new_states, dtype=tf.float32)

		with tf.GradientTape(persistent=True) as tape:
			target_actions = self.target_actor(states_)
			target_actions = target_actions + tf.clip_by_value(np.random.normal(scale=0.2), -0.5, 0.5)

			target_actions = tf.clip_by_value(target_actions, self.min_action, self.max_action)

			q1 = self.critic_1(states, actions)
			q2 = self.critic_2(states, actions)
			q1_ = self.target_critic_1(states_, target_actions)
			q2_ = self.target_critic_2(states_, target_actions)
	 
	 		# Making 1 D for neural network
			q1 = tf.squeeze(q1, 1)
			q2 = tf.squeeze(q2, 1)

			q1_ = tf.squeeze(q1_, 1)
			q2_ = tf.squeeze(q2_, 1)


			critic_value_ = tf.math.minimum(q1_, q2_)

			target = rewards + self.gamma*critic_value_*(1-dones)
			critic_1_loss = keras.losses.MSE(target, q1)
			critic_2_loss = keras.losses.MSE(target, q2)

		critic_1_gradient = tape.gradient(critic_1_loss, self.critic_1.trainable_variables)
		critic_2_gradient = tape.gradient(critic_2_loss, self.critic_2.trainable_variables)

		self.critic_1.optimizer.apply_gradients(zip(critic_1_gradient, self.critic_1.trainable_variables))
		self.critic_2.optimizer.apply_gradients(zip(critic_2_gradient, self.critic_2.trainable_variables))

		self.learn_step_cntr += 1

		if self.learn_step_cntr % self.update_actor_iter != 0:
			return

		with tf.GradientTape() as tape:
			new_actions = self.actor(states)
			critic_1_value = self.critic_1(states, new_actions)
			actor_loss = -tf.math.reduce_mean(critic_1_value)

		actor_gradient = tape.gradient(actor_loss, self.actor.trainable_variables)
		self.actor.optimizer.apply_gradients(zip(actor_gradient, self.actor.trainable_variables))

		self.update_network_parameters()

	def update_network_parameters(self, tau=None):
		if tau is None:
			tau = self.tau

		weights = []
		targets = self.target_actor.weights
		for i, weight in enumerate(self.actor.weights):
			weights.append(weight * tau + targets[i]*(1-tau))

		self.target_actor.set_weights(weights)

		weights = []
		targets = self.target_critic_1.weights
		for i, weight in enumerate(self.critic_1.weights):
			weights.append(weight * tau + targets[i]*(1-tau))

		self.target_critic_1.set_weights(weights)

		weights = []
		targets = self.target_critic_2.weights
		for i, weight in enumerate(self.critic_2.weights):
			weights.append(weight * tau + targets[i]*(1-tau))

		self.target_critic_2.set_weights(weights)

	def save_models(self):
		print('Saving models.......')
		self.actor.save_weights(self.actor.check_pt_file)
		self.target_actor.save_weights(self.target_actor.check_pt_file)
		self.critic_1.save_weights(self.critic_1.check_pt_file)
		self.critic_2.save_weights(self.critic_2.check_pt_file)
		self.target_critic_1.save_weights(self.target_critic_1.check_pt_file)
		self.target_critic_2.save_weights(self.target_critic_2.check_pt_file)

	def load_models(self):
		print('Loading models......')
		self.actor.load_weights(self.actor.check_pt_file)
		self.target_actor.load_weights(self.target_actor.check_pt_file)
		self.critic_1.load_weights(self.critic_1.check_pt_file)
		self.critic_2.load_weights(self.critic_2.check_pt_file)
		self.target_critic_1.load_weights(self.target_critic_1.check_pt_file)
		self.target_critic_2.load_weights(self.target_critic_2.check_pt_file)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import gym

render_every = 30

if __name__ == '__main__':
	# env = gym.make('BipedalWalker-v3')
	# video_writer_fourcc = cv2.VideoWriter_fourcc(*'XVID')
	# Creating video writer to save the progress of agent
	# video = cv2.VideoWriter('agent_behaviour.avi', video_writer_fourcc, 30, (400, 400))

	# env = tbot_2d_env()
	env = gym.make("BipedalWalker-v3", render_mode="rgb_array")
	env = gym.wrappers.RecordVideo(env, 'video', episode_trigger=lambda x: x % render_every == 0)
	agent = Agent(agent_lr=0.0005, critic_lr=0.001,
			input_dims=env.observation_space.shape, tau=0.005,
			env=env, batch_size=100, layer1_size=512, layer2_size=256, layer3_size=128,
			n_actions=env.action_space.shape[0])
	# agent.load_models()
	n_games = 3000

	best_score = -999
	score_history = []

	#agent.load_models()

	for i in range(n_games):
		observation = env.reset()
		# print(observation)
		observation = observation
		done = False
		score = 0
		while not done:
			action = agent.nw_action(observation)
			# print("Action is: ", action)
			observation_, reward, done, _ = env.step(action)
	 
			agent.remember(observation, action, reward, observation_, done)
			agent.learn()
	 
			observation = observation_
			score += reward

		score_history.append(score)
		avg_score = np.mean(score_history[-100:])

		if avg_score > best_score:
			best_score = avg_score
			agent.save_models()

		# if agent.ep_greedy < 0.995 and (i%20) == 0:
		# 	agent.ep_greedy += 0.005

		print('episode', i, 'score %.2f' % score, 'average score %.2f' % avg_score)

	# x = [i+1 for i in range(n_games)]

	video.release()
	# plot_learning_curve(x, score_history, filename)
	avg_rn = np.zeros(len(score_history))
	last_n_eps = 100
	x_axis = [j+1 for j in range(len(score_history))]
	for i in range(len(avg_rn)):
		avg_rn[i] = np.mean(score_history[max(0, i-last_n_eps):(i+1)])
	plt.plot(x_axis, score_history)
	# plt.title('Plot of running average of previous {} scores'.format(last_n_eps))
	plt.title('Plot of scores w.r.t. episodes for {} episodes'.format(len(score_history)))
	# plt.savefig(filename)
	plt.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


... saving models ...
episode 0 score -94.7 average score -94.7
episode 1 score -124.2 average score -109.5
episode 2 score -112.5 average score -110.5
episode 3 score -104.7 average score -109.1
episode 4 score -115.0 average score -110.2
episode 5 score -114.2 average score -110.9
episode 6 score -119.7 average score -112.2
episode 7 score -144.1 average score -116.1
episode 8 score -139.9 average score -118.8
episode 9 score -112.5 average score -118.2
episode 10 score -113.6 average score -117.7
episode 11 score -109.8 average score -117.1
episode 12 score -176.1 average score -121.6
episode 13 score -137.7 average score -122.8
episode 14 score -114.5 average score -122.2
episode 15 score -108.9 average score -121.4
episode 16 score -123.6 average score -121.5
episode 17 score -99.7 average score -120.3
episode 18 score -101.1 average score -119.3
episode 19 score -118.1 average score -119.2
episode 20 score -103.2 average score -118.5
episode 21 score -103.0 average score -117.8
e

/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:288: UserWarning: WARN: Tried to pass invalid video frame, marking as broken: Your frame has shape (400, 599, 3), but the VideoRecorder is configured for shape (400, 600, 3).
  logger.warn("Tried to pass invalid video frame, marking as broken: %s", e)


episode 30 score -128.0 average score -114.3
episode 31 score -135.5 average score -115.0
episode 32 score -143.1 average score -115.8
episode 33 score -101.2 average score -115.4
episode 34 score -100.9 average score -115.0
episode 35 score -102.3 average score -114.6
episode 36 score -105.4 average score -114.4
episode 37 score -104.8 average score -114.1
episode 38 score -104.3 average score -113.9
episode 39 score -107.9 average score -113.7
episode 40 score -119.6 average score -113.9
episode 41 score -148.0 average score -114.7
episode 42 score -103.8 average score -114.4
episode 43 score -109.0 average score -114.3
episode 44 score -141.8 average score -114.9
episode 45 score -138.9 average score -115.4
episode 46 score -122.2 average score -115.6
episode 47 score -105.6 average score -115.4
episode 48 score -109.4 average score -115.3
episode 49 score -96.9 average score -114.9
episode 50 score -100.3 average score -114.6
episode 51 score -110.2 average score -114.5
episode 52 

/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:288: UserWarning: WARN: Tried to pass invalid video frame, marking as broken: Your frame has shape (400, 599, 3), but the VideoRecorder is configured for shape (400, 600, 3).
  logger.warn("Tried to pass invalid video frame, marking as broken: %s", e)


episode 120 score -115.9 average score -109.8
episode 121 score -96.1 average score -109.7
episode 122 score -109.8 average score -109.8
episode 123 score -106.8 average score -109.8
episode 124 score -107.8 average score -109.9
episode 125 score -105.8 average score -109.9
episode 126 score -106.4 average score -110.0
episode 127 score -97.2 average score -109.9
episode 128 score -105.5 average score -109.9
episode 129 score -104.0 average score -109.9
episode 130 score -107.4 average score -109.7
episode 131 score -114.4 average score -109.5
episode 132 score -103.5 average score -109.1
episode 133 score -105.3 average score -109.1
episode 134 score -103.6 average score -109.1
episode 135 score -103.7 average score -109.1
episode 136 score -104.3 average score -109.1
episode 137 score -111.8 average score -109.2
episode 138 score -102.3 average score -109.2
episode 139 score -101.2 average score -109.1
episode 140 score -102.0 average score -108.9
episode 141 score -115.3 average sco

/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:288: UserWarning: WARN: Tried to pass invalid video frame, marking as broken: Your frame has shape (400, 599, 3), but the VideoRecorder is configured for shape (400, 600, 3).
  logger.warn("Tried to pass invalid video frame, marking as broken: %s", e)


episode 180 score -145.2 average score -110.9
episode 181 score -104.6 average score -110.8
episode 182 score -104.9 average score -110.8
episode 183 score -126.1 average score -111.0
episode 184 score -96.4 average score -110.9
episode 185 score -108.2 average score -110.8
episode 186 score -100.3 average score -110.7
episode 187 score -141.3 average score -111.1
episode 188 score -188.4 average score -112.0
episode 189 score -103.0 average score -112.0
episode 190 score -110.1 average score -112.0
episode 191 score -141.4 average score -112.3
episode 192 score -108.8 average score -112.4
episode 193 score -103.2 average score -112.3
episode 194 score -112.4 average score -112.4
episode 195 score -112.5 average score -112.2
episode 196 score -94.7 average score -111.9
episode 197 score -94.8 average score -111.7
episode 198 score 25.3 average score -110.4
episode 199 score -34.6 average score -109.7
episode 200 score -79.1 average score -109.5
episode 201 score -92.6 average score -10